# Real-time Implementation and Automation with Oanda 

--------------------------------------------------------------------------------------------------------------------

_Disclaimer: <br>
The following illustrative examples are for general information and educational purposes only. <br>
It is neither investment advice nor a recommendation to trade, invest or take whatsoever actions.<br>
The below code should only be used in combination with an Oanda Practice/Demo Account and NOT with a Live Trading Account._

------------------------------------------------------------------------------------

## Recap: Historical Data, real-time Data and Orders

In [2]:
import pandas as pd
import tpqoa
import warnings
warnings.filterwarnings('ignore')

In [3]:
api = tpqoa.tpqoa("oanda.cfg")

In [4]:
api.get_instruments()

[('AUD/CAD', 'AUD_CAD'),
 ('AUD/CHF', 'AUD_CHF'),
 ('AUD/HKD', 'AUD_HKD'),
 ('AUD/JPY', 'AUD_JPY'),
 ('AUD/NZD', 'AUD_NZD'),
 ('AUD/SGD', 'AUD_SGD'),
 ('AUD/USD', 'AUD_USD'),
 ('Australia 200', 'AU200_AUD'),
 ('Brent Crude Oil', 'BCO_USD'),
 ('Bund', 'DE10YB_EUR'),
 ('CAD/CHF', 'CAD_CHF'),
 ('CAD/HKD', 'CAD_HKD'),
 ('CAD/JPY', 'CAD_JPY'),
 ('CAD/SGD', 'CAD_SGD'),
 ('CHF/HKD', 'CHF_HKD'),
 ('CHF/JPY', 'CHF_JPY'),
 ('CHF/ZAR', 'CHF_ZAR'),
 ('China A50', 'CN50_USD'),
 ('China H Shares', 'CHINAH_HKD'),
 ('Copper', 'XCU_USD'),
 ('Corn', 'CORN_USD'),
 ('EUR/AUD', 'EUR_AUD'),
 ('EUR/CAD', 'EUR_CAD'),
 ('EUR/CHF', 'EUR_CHF'),
 ('EUR/CZK', 'EUR_CZK'),
 ('EUR/DKK', 'EUR_DKK'),
 ('EUR/GBP', 'EUR_GBP'),
 ('EUR/HKD', 'EUR_HKD'),
 ('EUR/HUF', 'EUR_HUF'),
 ('EUR/JPY', 'EUR_JPY'),
 ('EUR/NOK', 'EUR_NOK'),
 ('EUR/NZD', 'EUR_NZD'),
 ('EUR/PLN', 'EUR_PLN'),
 ('EUR/SEK', 'EUR_SEK'),
 ('EUR/SGD', 'EUR_SGD'),
 ('EUR/TRY', 'EUR_TRY'),
 ('EUR/USD', 'EUR_USD'),
 ('EUR/ZAR', 'EUR_ZAR'),
 ('Europe 50', 'EU50_EUR

__Historical Data__

In [5]:
api.get_history(instrument = "EUR_USD", start = "2024-09-24", end = "2024-09-26",
                granularity = "M1", price = "M", localize = False) # S5 drops a Future Warning which can be ignored for now

,o,h,l,c,volume,complete
time,,,,,,
2024-09-24 00:00:00+00:00,1.11109,1.11120,1.11106,1.11120,63,True
2024-09-24 00:01:00+00:00,1.11119,1.11122,1.11105,1.11105,47,True
2024-09-24 00:02:00+00:00,1.11106,1.11106,1.11090,1.11101,44,True
2024-09-24 00:03:00+00:00,1.11098,1.11099,1.11084,1.11086,72,True
2024-09-24 00:04:00+00:00,1.11086,1.11092,1.11084,1.11092,76,True
...,...,...,...,...,...,...
2024-09-25 23:55:00+00:00,1.11306,1.11308,1.11306,1.11308,17,True
2024-09-25 23:56:00+00:00,1.11308,1.11312,1.11306,1.11312,10,True
2024-09-25 23:57:00+00:00,1.11312,1.11313,1.11305,1.11306,11,True


__Streaming ticks / real-time Data__

In [6]:
api.stream_data("EUR_USD", stop = 20) 

V20Timeout: v20 REST request to https://stream-fxpractice.oanda.com:443/v3/accounts/101-004-36234373-001/pricing/stream has timed out (read)

__Orders & Trades__

In [ ]:
api.create_order(instrument = "EUR_USD", units = 100000)

In [ ]:
api.get_positions()

In [ ]:
api.create_order(instrument = "EUR_USD", units = -100000)

In [ ]:
api.get_account_summary()

In [ ]:
api.get_transactions()

In [ ]:
api.print_transactions()

In [ ]:
order = api.create_order(instrument = "EUR_USD", units = 100000, suppress = True, ret = True)

In [ ]:
order

In [ ]:
order["price"]

In [ ]:
order2 = api.create_order(instrument = "EUR_USD", units = -100000, suppress = True, ret = True)

In [ ]:
order2 

In [ ]:
order2["price"]

In [ ]:
float(order2["pl"])

In [ ]:
order2["time"]

In [ ]:
order2["units"]

In [ ]:
order2["id"]

## Preview: A Trader Class live in action

In [ ]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timezone, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = [] # NEW
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.now(timezone.utc).replace(tzinfo=None)
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.now(timezone.utc)) - self.last_bar < self.bar_length:
                break
            
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # collect and store tick data
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) # new with pd.concat()
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar >= self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = pd.concat([self.raw_data, self.tick_data.resample(self.bar_length, 
                                                                          label="right").last().ffill().iloc[:-1]]) 
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
        
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")  # NEW
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")  # NEW
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  # NEW
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  # NEW
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")  # NEW
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")  # NEW
            self.position = 0
    
    def report_trade(self, order, going):  # NEW
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
  

Simple Contrarian: Bar_lenght = 1min | Window = 1 (1 minute)

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", bar_length = "1min", window = 1, units = 100000)

In [ ]:
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 200)
if trader.position != 0: # if we have a final open position
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0

Simple Contrarian: Bar_lenght = 1min | Window = 60 (1 hour)

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", bar_length = "1min", window = 60, units = 100000)

In [ ]:
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 200)
if trader.position != 0: # if we have a final open position
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0

## Collecting and storing real-time tick data

In [ ]:
import pandas as pd
import tpqoa

In [ ]:
api = tpqoa.tpqoa("oanda.cfg")

In [ ]:
api.stream_data("EUR_USD", stop = 10)

In [ ]:
# two steps: 1) Retrieve tick data 2) call on_sucess() method to print tick data
api.stream_data("EUR_USD")

In [ ]:
# called by stream_data() when new data is retrieved 
api.on_success()

In [ ]:
class CloneClass(tpqoa.tpqoa):
    
    def on_success(self, time, bid, ask):
        print("Time: {} | Bid: {} | Ask:{}".format(time, bid, ask))

In [ ]:
api = CloneClass("oanda.cfg")

In [ ]:
api.get_instruments()[:5]

In [ ]:
api.stream_data("EUR_USD", stop = 5)

## Storing and resampling real-time tick data (Part 1)

In [ ]:
import pandas as pd
import tpqoa

In [ ]:
class GetTickData(tpqoa.tpqoa):
    
    def __init__(self, config_file):
        super().__init__(config_file)
        self.tick_data = pd.DataFrame()
    
    def on_success(self, time, bid, ask):
        print(time, bid, ask)
        df = pd.DataFrame({"bid":bid, "ask":ask, "mid":(ask + bid)/2}, 
                          index = [pd.to_datetime(time)])
        #self.tick_data = self.tick_data.append(df) old -> append() deprecated
        self.tick_data = pd.concat([self.tick_data, df]) # new with pd.concat()

In [ ]:
td = GetTickData("oanda.cfg")

In [ ]:
td.stream_data("EUR_USD", stop = 10)

In [ ]:
td.tick_data

In [ ]:
td.tick_data.resample("1s", label = "right").last().ffill() # resample to 1s bars

In [ ]:
td = GetTickData("oanda.cfg")

In [ ]:
td.stream_data("EUR_USD", stop = 100)

In [ ]:
td.tick_data.resample("10s", label = "right").last()#.ffill()

## Storing and resampling real-time tick data (Part 2)

In [ ]:
import pandas as pd
import tpqoa

In [ ]:
class GetTickData(tpqoa.tpqoa):
    
    def __init__(self, config_file, bar_length):
        super().__init__(config_file)
        self.bar_length = bar_length
        self.tick_data = pd.DataFrame()
    
    def on_success(self, time, bid, ask):
        print(time, bid, ask)
        df = pd.DataFrame({"bid":bid, "ask":ask, "mid":(ask + bid)/2}, 
                          index = [pd.to_datetime(time)])
        self.tick_data = pd.concat([self.tick_data, df]) # new with pd.concat()
        self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]

In [ ]:
td = GetTickData("oanda.cfg", "5s")

In [ ]:
td.stream_data("EUR_USD", stop = 20)

In [ ]:
td.data

In [ ]:
td.tick_data

## Storing and resampling real-time tick data (Part 3)

In [ ]:
import pandas as pd
import tpqoa

In [ ]:
class ConTrader(tpqoa.tpqoa): # Ultimate Goal: Implementing a simple Contrarian Strategy
    
    def __init__(self, config_file, instrument, bar_length):
        super().__init__(config_file)
        self.instrument = instrument # define instrument
        self.bar_length = bar_length
        self.tick_data = pd.DataFrame()
    
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ") # Print running Tick number
        
        # collect and store tick data
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [pd.to_datetime(time)]) # mid price only
        self.tick_data = pd.concat([self.tick_data, df]) # new with pd.concat()
        
        self.resample_and_join() # NEW
        
    def resample_and_join(self): # NEW
        self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]

In [ ]:
for i in range(10):
    print(i, end = " ")  

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "5s")

In [ ]:
trader.stream_data("EUR_USD", stop = 20)

In [ ]:
trader.data

In [ ]:
trader.tick_data

## Storing and resampling real-time tick data (Part 4)

__When should we resample?__ -> Only when a __full new bar__ is available!<br>
-> Whenever there are more than 5s (bar_length) between the __most recent tick__ and __last full bar__. 

In [ ]:
import pandas as pd
import tpqoa
from datetime import datetime, timezone  # timezone added (Python 3.12)

In [ ]:
class ConTrader(tpqoa.tpqoa):
    
    def __init__(self, config_file, instrument, bar_length):
        super().__init__(config_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) # Pandas Timedelta Object
        self.tick_data = pd.DataFrame()
        self.last_bar = pd.to_datetime(datetime.now(timezone.utc)) # UTC time at instantiation (NEW Python 3.12)
    
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # collect and store tick data
        recent_tick = pd.to_datetime(time) # Pandas Timestamp Object
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) # new with pd.concat()
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
    
    def resample_and_join(self):
        self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]
        self.last_bar = self.data.index[-1]  # update time of last full bar
    

In [ ]:
datetime.utcnow() # old

In [ ]:
# pd.to_datetime(datetime.utcnow()).tz_localize("UTC") # old

In [ ]:
datetime.now(timezone.utc) # new Python 3.12

In [ ]:
pd.to_datetime(datetime.now(timezone.utc)) # new

In [ ]:
pd.to_timedelta("5s")

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "5s")
trader.stream_data(trader.instrument, stop = 20)

In [ ]:
trader.data

In [ ]:
trader.tick_data

## Storing and resampling real-time tick data (Part 5)

In [ ]:
import pandas as pd
import tpqoa
from datetime import datetime, timezone  # timezone added (Python 3.12)

Goal: Once we have added a new full bar...<br>
- delete the correpsonding ticks from self.tick_data
- only keep the latest tick (next bar)

(Recap: receiving the first tick of the next bar is the signal to resample)

In [ ]:
class ConTrader(tpqoa.tpqoa):
    
    def __init__(self, config_file, instrument, bar_length):
        super().__init__(config_file)
        self.instrument = instrument 
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.data = pd.DataFrame() # NEW
        self.last_bar = pd.to_datetime(datetime.now(timezone.utc)) # UTC time at instantiation (NEW Python 3.12)
    
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ") 
        
        # collect and store tick data
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick]) 
        self.tick_data = pd.concat([self.tick_data, df]) # new with pd.concat()
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
    
    def resample_and_join(self):
        #self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]
        # NEW: append the most recent ticks (resampled) to self.data with concat
        self.data = pd.concat([self.data, self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1]])
        
        self.tick_data = self.tick_data.iloc[-1:] # NEW: only keep the latest tick (next bar)
        self.last_bar = self.data.index[-1] 

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "5s")
trader.stream_data(trader.instrument, stop = 20)

In [ ]:
trader.data

In [ ]:
trader.tick_data # only most recent ticks since last full bar

## Working with historical data and real-time tick data (Part 1)

In [ ]:
import pandas as pd
import tpqoa
from datetime import datetime, timezone, timedelta 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
now = datetime.utcnow() # old
now

In [ ]:
now = datetime.now(timezone.utc).replace(tzinfo=None) # new (Python 3.12)
now

In [ ]:
now = now - timedelta(microseconds = now.microsecond)
now

In [ ]:
yesterday = now - timedelta(days = 1)
yesterday

In [ ]:
api = tpqoa.tpqoa("oanda.cfg")

In [ ]:
instrument = "EUR_USD"

In [ ]:
df = api.get_history(instrument = instrument, start = yesterday, end = now,
                     granularity = "S5", price = "M", localize = False)
df

In [ ]:
df = df.c.dropna().to_frame()
df

In [ ]:
df.rename(columns = {"c":instrument}, inplace = True)
df

In [ ]:
df = df.resample("1min", label = "right").last().dropna().iloc[:-1]
df

In [ ]:
last_bar = df.index[-1]
last_bar

__Troubleshooting__

Don´t pass localized datetime objects to api.get_history()!

In [ ]:
now = datetime.now(timezone.utc).replace(tzinfo=None) # new (Python 3.12)
now

In [ ]:
now = now - timedelta(microseconds = now.microsecond)
now

In [ ]:
yesterday = now - timedelta(days = 1)
yesterday

In [ ]:
df = api.get_history(instrument = instrument, start = yesterday, end = now,
                     granularity = "S5", price = "M", localize = False)
df

## Working with historical data and real-time tick data (Part 2)

In [ ]:
import pandas as pd
import tpqoa
from datetime import datetime, timezone, timedelta 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.data = None # first defined in get_most_recent()
        self.last_bar = None # first defined in get_most_recent()
    
    def get_most_recent(self, days = 5): # NEW
        now = datetime.now(timezone.utc).replace(tzinfo=None) # new (Python 3.12)
        now = now - timedelta(microseconds = now.microsecond)
        past = now - timedelta(days = days)
        df = self.get_history(instrument = self.instrument, start = past, end = now,
                              granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
        df.rename(columns = {"c":self.instrument}, inplace = True)
        df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
        self.data = df.copy() # first defined
        self.last_bar = self.data.index[-1] # first defined
            
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # collect and store tick data
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) 
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
            
    def resample_and_join(self):
        self.data = pd.concat([self.data, self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1]])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.data.index[-1]

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1m")

In [ ]:
print(datetime.now(timezone.utc).replace(tzinfo=None)) # new (Python 3.12)
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 50)

In [ ]:
trader.data.tail(10)

In [ ]:
trader.tick_data

## Working with historical data and real-time tick data (Part 3)

In [ ]:
import pandas as pd
import tpqoa
from datetime import datetime, timezone, timedelta 
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.data = None 
        self.last_bar = None
        
        self.count = 0 # for demonstration purposes only
    
    def get_most_recent(self, days = 5):
        while True: # repeat until we get all historical bars
            self.count += 1 # for demonstration purposes only
            time.sleep(2)
            now = datetime.now(timezone.utc).replace(tzinfo=None) 
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.data = df.copy()
            self.last_bar = self.data.index[-1]
            # accept, if less than [bar_lenght] has elapsed since the last full historical bar and now
            if pd.to_datetime(datetime.now(timezone.utc)) - self.last_bar < self.bar_length: # adjusted to Python 3.12
                break
            
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # collect and store tick data
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) # new with pd.concat()
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar >= self.bar_length:
            self.resample_and_join()
            
    def resample_and_join(self):
        self.data = pd.concat([self.data, self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1]])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.data.index[-1]

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1m")

In [ ]:
print(datetime.now(timezone.utc).replace(tzinfo=None)) # new (Python 3.12)
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 50)

In [ ]:
trader.data#.tail(10)

In [ ]:
trader.count

## Defining a Simple Contrarian Trading Strategy

In [ ]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timezone, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None # NEW
        self.data = None 
        self.last_bar = None
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.now(timezone.utc).replace(tzinfo=None) 
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy() # raw!
            self.last_bar = self.raw_data.index[-1] # raw!
            if pd.to_datetime(datetime.now(timezone.utc)) - self.last_bar < self.bar_length:
                break
            
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # collect and store tick data
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) # new with pd.concat()
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar >= self.bar_length:
            self.resample_and_join()
            self.define_strategy() # Prepare Data / Strategy Features
            
    def resample_and_join(self):
        self.raw_data = pd.concat([self.raw_data, self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1]]) # raw!
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1] # raw! 
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy() # self.raw_data new!
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy() # first defined here


In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1min", window = 1)

In [ ]:
print(datetime.now(timezone.utc).replace(tzinfo=None))
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 100)

In [ ]:
trader.raw_data

In [ ]:
trader.data

## Placing Orders and Executing Trades

In [ ]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timezone, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units # NEW
        self.position = 0 # NEW
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.now(timezone.utc).replace(tzinfo=None) 
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.now(timezone.utc)) - self.last_bar < self.bar_length:
                break
            
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # collect and store tick data
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) # new with pd.concat()
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar >= self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades() # NEW!
            
    def resample_and_join(self):
        self.raw_data = pd.concat([self.raw_data, self.tick_data.resample(self.bar_length, 
                                                                          label="right").last().ffill().iloc[:-1]]) 
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
        
    def execute_trades(self): # NEW!
        if self.data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                print("GOING LONG")
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                print("GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: # if position is short -> go/stay short
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                print("GOING SHORT")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                print("GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0:  # if position is neutral -> go/stay neutral
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                print("GOING NEUTRAL")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                print("GOING NEUTRAL")
            self.position = 0

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1min", window = 1, units = 100000)

In [ ]:
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 100)

In [ ]:
trader.position

In [ ]:
trader.units

In [ ]:
trader.get_positions()

In [ ]:
if trader.position != 0: # if we have a final open position
    order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.position = 0

In [ ]:
trader.data


## Trade Monitoring and Reporting

In [ ]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timezone, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = [] # NEW
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.now(timezone.utc).replace(tzinfo=None)
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.now(timezone.utc)) - self.last_bar < self.bar_length:
                break
            
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # collect and store tick data
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) # new with pd.concat()
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar >= self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = pd.concat([self.raw_data, self.tick_data.resample(self.bar_length, 
                                                                          label="right").last().ffill().iloc[:-1]]) 
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
        
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")  # NEW
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")  # NEW
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  # NEW
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  # NEW
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")  # NEW
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")  # NEW
            self.position = 0
    
    def report_trade(self, order, going):  # NEW
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
    

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1min", window = 1, units = 100000)

In [ ]:
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 150)
if trader.position != 0: # if we have a final open position
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0

In [ ]:
trader.get_positions()

In [ ]:
trader.profits

In [ ]:
sum(trader.profits)

In [ ]:
trader.data.tail(10)

In [ ]:
trader.tick_data

## Trading other Strategies - Coding Challenge

In [ ]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timezone, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

__Strategy 1__: SMA 50/200 (minutes) Crossover (200 ticks)

## Stop here if you don´t want to see the solution!

###############################################################

In [ ]:
class SMATrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, SMA_S, SMA_L, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = []
        
        #*****************add strategy-specific attributes here******************
        self.SMA_S = SMA_S
        self.SMA_L = SMA_L
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.now(timezone.utc).replace(tzinfo=None)
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.now(timezone.utc)) - self.last_bar < self.bar_length:
                break
                
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) # new with pd.concat()
        
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
    
    def resample_and_join(self):
        self.raw_data = pd.concat([self.raw_data, self.tick_data.resample(self.bar_length, 
                                                                          label="right").last().ffill().iloc[:-1]])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
    
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["SMA_S"] = df[self.instrument].rolling(self.SMA_S).mean()
        df["SMA_L"] = df[self.instrument].rolling(self.SMA_L).mean()
        df["position"] = np.where(df["SMA_S"] > df["SMA_L"], 1, -1)
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0
    
    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
    

In [ ]:
trader = SMATrader("oanda.cfg", "EUR_USD", "1min", SMA_S = 50, SMA_L = 200, units = 100000)

In [ ]:
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 200)
if trader.position != 0: # if we have a final open position
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0

In [ ]:
trader.data#.tail(20)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
trader.data.tail(30).plot(figsize = (12, 8), secondary_y = "position")
plt.show()

__Strategy 2__: Bollinger Bands SMA 20 (minutes) / 1 Standard Deviation (200 ticks) 

## Stop here if you don´t want to see the solution!

###############################################################

In [ ]:
class BollTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, SMA, dev, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = []
        
        #*****************add strategy-specific attributes here******************
        self.SMA = SMA
        self.dev = dev
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.now(timezone.utc).replace(tzinfo=None)
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.now(timezone.utc)) - self.last_bar < self.bar_length:
                break
                
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) # new with pd.concat()
        
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
    
    def resample_and_join(self):
        self.raw_data = pd.concat([self.raw_data, self.tick_data.resample(self.bar_length, 
                                                                          label="right").last().ffill().iloc[:-1]])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
    
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["SMA"] = df[self.instrument].rolling(self.SMA).mean()
        df["Lower"] = df["SMA"] - df[self.instrument].rolling(self.SMA).std() * self.dev
        df["Upper"] = df["SMA"] + df[self.instrument].rolling(self.SMA).std() * self.dev
        df["distance"] = df[self.instrument] - df.SMA
        df["position"] = np.where(df[self.instrument] < df.Lower, 1, np.nan)
        df["position"] = np.where(df[self.instrument] > df.Upper, -1, df["position"])
        df["position"] = np.where(df.distance * df.distance.shift(1) < 0, 0, df["position"])
        df["position"] = df.position.ffill().fillna(0)
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0
    
    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
 

In [ ]:
trader = BollTrader("oanda.cfg", "EUR_USD", "1min", SMA = 20, dev = 1, units = 100000)

In [ ]:
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 200)
if trader.position != 0: # if we have a final open position
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0

In [ ]:
trader.data

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
trader.data.tail(20)[["EUR_USD", "SMA", "Lower", "Upper"]].plot(figsize = (12, 8))
plt.show()

## Machine Learning Strategies (1) - Model Fitting

In [112]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier # added (from sklearn v. 1.7)

In [113]:
data = pd.read_csv("five_minute.csv", parse_dates = ["time"], index_col = "time")

In [114]:
data

,price
time,
2019-01-01 22:00:00+00:00,1.146580
2019-01-01 22:05:00+00:00,1.146350
2019-01-01 22:10:00+00:00,1.146320
2019-01-01 22:15:00+00:00,1.146320
2019-01-01 22:20:00+00:00,1.146530
...,...
2019-12-30 23:35:00+00:00,1.120180
2019-12-30 23:40:00+00:00,1.120210
2019-12-30 23:45:00+00:00,1.120295


In [115]:
data["returns"] = np.log(data.div(data.shift(1)))

In [116]:
data.dropna(inplace = True)

In [117]:
data["direction"] = np.sign(data.returns)

In [118]:
data

,price,returns,direction
time,,,
2019-01-01 22:05:00+00:00,1.146350,-0.000201,-1.0
2019-01-01 22:10:00+00:00,1.146320,-0.000026,-1.0
2019-01-01 22:15:00+00:00,1.146320,0.000000,0.0
2019-01-01 22:20:00+00:00,1.146530,0.000183,1.0
2019-01-01 22:25:00+00:00,1.146475,-0.000048,-1.0
...,...,...,...
2019-12-30 23:35:00+00:00,1.120180,-0.000004,-1.0
2019-12-30 23:40:00+00:00,1.120210,0.000027,1.0
2019-12-30 23:45:00+00:00,1.120295,0.000076,1.0


In [119]:
lags = 2

In [120]:
cols = []
for lag in range(1, lags + 1):
    col = "lag{}".format(lag)
    data[col] = data.returns.shift(lag)
    cols.append(col)
data.dropna(inplace = True)

++++++++++++++++++++++++++++

__Scaling/Standardizing Features (new)__

In [121]:
means = data[cols].mean()
means

lag1   -3.121233e-07
lag2   -3.146025e-07
dtype: float64

In [122]:
stand_devs = data[cols].std()
stand_devs

lag1    0.000199
lag2    0.000199
dtype: float64

In [123]:
data[cols] = (data[cols]-means) / stand_devs
data

,price,returns,direction,lag1,lag2
time,,,,,
2019-01-01 22:15:00+00:00,1.146320,0.000000,0.0,-0.130239,-1.008842
2019-01-01 22:20:00+00:00,1.146530,0.000183,1.0,0.001572,-0.130225
2019-01-01 22:25:00+00:00,1.146475,-0.000048,-1.0,0.924174,0.001585
2019-01-01 22:30:00+00:00,1.146455,-0.000017,-1.0,-0.240045,0.924180
2019-01-01 22:35:00+00:00,1.146455,0.000000,0.0,-0.086292,-0.240031
...,...,...,...,...,...
2019-12-30 23:35:00+00:00,1.120180,-0.000004,-1.0,-0.560428,-0.088329
2019-12-30 23:40:00+00:00,1.120210,0.000027,1.0,-0.020909,-0.560412
2019-12-30 23:45:00+00:00,1.120295,0.000076,1.0,0.136458,-0.020897


+++++++++++++++++++++++++++

In [ ]:
# lm = LogisticRegression(C = 1e6, max_iter = 100000, multi_class = "ovr") # old

In [124]:
lm = OneVsRestClassifier(LogisticRegression(C = 1e6, max_iter = 100000)) # new (from sklearn v. 1.7)

In [125]:
lm.fit(data[cols], data.direction)

,estimator,LogisticRegre...x_iter=100000)
,n_jobs,None
,verbose,0
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1000000.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None


In [126]:
lm

,estimator,LogisticRegre...x_iter=100000)
,n_jobs,None
,verbose,0
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1000000.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None


__In-Sample "Prediction"__

In [127]:
data["pred"] = lm.predict(data[cols])

In [128]:
data

,price,returns,direction,lag1,lag2,pred
time,,,,,,
2019-01-01 22:15:00+00:00,1.146320,0.000000,0.0,-0.130239,-1.008842,1.0
2019-01-01 22:20:00+00:00,1.146530,0.000183,1.0,0.001572,-0.130225,1.0
2019-01-01 22:25:00+00:00,1.146475,-0.000048,-1.0,0.924174,0.001585,-1.0
2019-01-01 22:30:00+00:00,1.146455,-0.000017,-1.0,-0.240045,0.924180,1.0
2019-01-01 22:35:00+00:00,1.146455,0.000000,0.0,-0.086292,-0.240031,1.0
...,...,...,...,...,...,...
2019-12-30 23:35:00+00:00,1.120180,-0.000004,-1.0,-0.560428,-0.088329,1.0
2019-12-30 23:40:00+00:00,1.120210,0.000027,1.0,-0.020909,-0.560412,1.0
2019-12-30 23:45:00+00:00,1.120295,0.000076,1.0,0.136458,-0.020897,-1.0


In [129]:
hits = np.sign(data.direction * data.pred).value_counts()

In [130]:
hits

 1.0    37472
-1.0    34286
 0.0     1959
Name: count, dtype: int64

In [131]:
hit_ratio = hits[1.0] / sum(hits)
hit_ratio

np.float64(0.5083223679748227)

In [132]:
lm

,estimator,LogisticRegre...x_iter=100000)
,n_jobs,None
,verbose,0
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1000000.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None


__Saving the model__

In [133]:
import pickle

In [134]:
pickle.dump(lm, open("logreg.pkl", "wb"))

__Saving the parameters (mean, std)__ NEW

In [135]:
params = {"mu":means, "std":stand_devs}
params

{'mu': lag1   -3.121233e-07
 lag2   -3.146025e-07
 dtype: float64,
 'std': lag1    0.000199
 lag2    0.000199
 dtype: float64}

In [136]:
pickle.dump(params, open("params.pkl", "wb"))

## Machine Learning Strategies (2) - Implementation

In [137]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier 
import tpqoa
from datetime import datetime, timezone, timedelta
import time
import pickle
import warnings
warnings.filterwarnings('ignore')

In [138]:
class MLTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, lags, model, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = []
        
        #*****************add strategy-specific attributes here******************
        self.lags = lags
        self.model = model
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.now(timezone.utc).replace(tzinfo=None)
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.now(timezone.utc)) - self.last_bar < self.bar_length:
                break
                
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) # new with pd.concat()
        
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
    
    def resample_and_join(self):
        self.raw_data = pd.concat([self.raw_data, self.tick_data.resample(self.bar_length, 
                                                                          label="right").last().ffill().iloc[:-1]])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
    
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df = pd.concat([df, self.tick_data]) # new with pd.concat
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        cols = []
        for lag in range(1, self.lags + 1):
            col = "lag{}".format(lag)
            df[col] = df.returns.shift(lag)
            cols.append(col)
        df.dropna(inplace = True)
        
        df[cols] = (df[cols] - means) / stand_devs # newly added (scaling)
                
        df["position"] = lm.predict(df[cols])
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0
    
    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  

In [139]:
lm = pickle.load(open("logreg.pkl", "rb"))
lm

,estimator,LogisticRegre...x_iter=100000)
,n_jobs,None
,verbose,0
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1000000.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None


+++++++++++++++++++++++++++++++++++++++

__Loading the parameters (mean, std)__ NEW

In [140]:
params = pickle.load(open("params.pkl", "rb"))
params

{'mu': lag1   -3.121233e-07
 lag2   -3.146025e-07
 dtype: float64,
 'std': lag1    0.000199
 lag2    0.000199
 dtype: float64}

In [141]:
means = params["mu"]
stand_devs = params["std"]

In [142]:
means

lag1   -3.121233e-07
lag2   -3.146025e-07
dtype: float64

+++++++++++++++++++++++++++++++++++++++

In [143]:
trader = MLTrader("oanda.cfg", "EUR_USD", "5min", lags = 2, model = lm, units = 100000)

In [144]:
trader.model

,estimator,LogisticRegre...x_iter=100000)
,n_jobs,None
,verbose,0
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1000000.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None


In [146]:
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 300)
if trader.position != 0: # if we have a final open position
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 
----------------------------------------------------------------------------------------------------
2025-10-14T16:55:00.484938201Z | GOING LONG
2025-10-14T16:55:00.484938201Z | units = 100000.0 | price = 1.16114 | P&L = 0.0 | Cum P&L = 0.0
------

In [147]:
trader.data.tail(10)

,EUR_USD,returns,lag1,lag2,position
2025-10-14 16:15:00+00:00,1.160320,-0.000034,-2.081476,0.478870,1.0
2025-10-14 16:20:00+00:00,1.161260,0.000810,-0.172054,-2.081449,1.0
2025-10-14 16:25:00+00:00,1.160510,-0.000646,4.080207,-0.172041,-1.0
2025-10-14 16:30:00+00:00,1.160850,0.000293,-3.252392,4.080192,1.0
2025-10-14 16:35:00+00:00,1.160600,-0.000215,1.476963,-3.252357,-1.0
2025-10-14 16:40:00+00:00,1.161240,0.000551,-1.083232,1.476965,1.0
2025-10-14 16:45:00+00:00,1.161390,0.000129,2.778204,-1.083212,-1.0
2025-10-14 16:50:00+00:00,1.161100,-0.000250,0.652124,2.778197,-1.0
2025-10-14 16:55:00+00:00,1.161105,0.000004,-1.256237,0.652132,1.0
2025-10-14 16:55:00.279133232+00:00,1.161100,-0.000004,0.023261,-1.256216,1.0


In [148]:
trader.tick_data

,EUR_USD
2025-10-14 16:55:00.279133232+00:00,1.161100
2025-10-14 16:55:00.484938201+00:00,1.161100
2025-10-14 16:55:00.814318272+00:00,1.161105
2025-10-14 16:55:03.679342295+00:00,1.161100
2025-10-14 16:55:03.966755302+00:00,1.161090
...,...
2025-10-14 16:56:00.495520203+00:00,1.161210
2025-10-14 16:56:01.016443965+00:00,1.161265
2025-10-14 16:56:01.318960773+00:00,1.161260
2025-10-14 16:56:01.633163327+00:00,1.161260


## Importing a Trader Module / Class

In [149]:
import trader as tr

In [150]:
trader = tr.ConTrader("oanda.cfg", "EUR_USD", "1min", window = 1, units = 100000)
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 100)
if trader.position != 0: # if we have a final open position
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 
----------------------------------------------------------------------------------------------------
2025-10-14T17:01:00.214431027Z | GOING SHORT
2025-10-14T17:01:00.214431027Z | units = -100000.0 | price = 1.16093 | P&L = 0.0 | Cum P&L = 0.0
----------------------------------------------------------------------------------------------------

32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 
----------------------------------------------------------------------------------------------------
2025-10-14T17:01:37.637506327Z | GOING NEUTRAL
2025-10-14T17:01:37.637506327Z | units = 100000.0 | price = 1.1611 | P&L = -12.8273 | Cum P&L = -12.8273
----------------------------------------------------------------------------------------------------



In [151]:
trader.data.tail(10)

,EUR_USD,returns,position
2025-10-14 16:52:00+00:00,1.161440,0.000155,-1.0
2025-10-14 16:53:00+00:00,1.161310,-0.000112,1.0
2025-10-14 16:54:00+00:00,1.161140,-0.000146,1.0
2025-10-14 16:55:00+00:00,1.161100,-0.000034,1.0
2025-10-14 16:56:00+00:00,1.161210,0.000095,-1.0
2025-10-14 16:57:00+00:00,1.161270,0.000052,-1.0
2025-10-14 16:58:00+00:00,1.161010,-0.000224,1.0
2025-10-14 16:59:00+00:00,1.160900,-0.000095,1.0
2025-10-14 17:00:00+00:00,1.160950,0.000043,-1.0
2025-10-14 17:01:00+00:00,1.160975,0.000022,-1.0


####################################################################################################

## How to Time/Schedule the End of a Trading Session (Simple Contrarian)

(you can find the corresponding Video for the following lines of code in __Section 26__: Cloud Deployment (AWS) | Scheduling Trading Sessions | Full Automation)

In [ ]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timezone, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# datetime.utcnow().time() # current utc time (old)

In [ ]:
datetime.now(timezone.utc).time() # new (Python 3.12)

In [ ]:
pd.to_datetime("17:30").time() # desired end of trading session (in utc time)

In [ ]:
if datetime.now(timezone.utc).time() >= pd.to_datetime("17:30").time():
    print("Stop the Trading Session!")

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = []
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.now(timezone.utc).replace(tzinfo=None)
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.now(timezone.utc)) - self.last_bar < self.bar_length:
                break
                
    def on_success(self, time, bid, ask):
        
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(time)
        
        # define stop
        if recent_tick.time() >= pd.to_datetime("13:00").time():
            self.stop_stream = True
        
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) # new with pd.concat()
        
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
    
    def resample_and_join(self):
        self.raw_data = pd.concat([self.raw_data, self.tick_data.resample(self.bar_length, 
                                                                          label="right").last().ffill().iloc[:-1]])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
    
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0:
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0
    
    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
    

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1min", window = 1, units = 100000)

In [ ]:
trader.get_most_recent()
trader.stream_data(trader.instrument) # no stop after n ticks!
if trader.position != 0: # if we have a final open position
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0